## **Pipelines & Transformers para datos numéricos y categóricos**

En esta notebook vamos a jugar con la creación de Pipelines y Transformers/Vectorizers para simplificar el desarrollo de un pipeline de procesamiento, que incluya desde el pre-procesamiento de los datos, hasta el entrenamiento y testing de un clasificador.


Como venimos trabajando, en la mayoría de las aplicaciones de Machine Learning, los datos con los que hay que trabajar no se encuentran en las condiciones óptimas para entrenar el "mejor" modelo posible.

Existen diversos pasos a realizar a las features, dependientes su tipo. Por ejemplo, puede haber un encoding para las features nominales o categóricas, escalado y normalización para features numéricas, y otras tantas alternativas de pre-procesamiento para el texto. Recordemos que los atributos textuales no son bien recibidos por los modelos a entrenar dado que los mismos esperan una representación numérica.

En las notebooks anteriores el procesamiento lo realizamos de forma "manual", es decir, creando algunos métodos para aplicar el pre-procesamiento a los datos de forma independiente de otras estructuras o tareas. Sin embargo, esta sepración e independencia puede resultar "tediosa". Si el objetivo último es entrenar un modelo, debemos recordar que hay que aplicar el pre-procesamiento tanto al training como al test, y luego a cada uno de los elementos que querramos evaluar con el modelo. Por otra parte, si quisieramos compartir nuestro modelo con otros, también tenemos que recordar compartir el pre-procesamiento de los datos aparte.

En este contexto, ``Scikit-learn`` nos provee de algunos mecanismos para simplificar la integración de las tareas y el proceso: los pipelines y los transformers, que permiten:

* Hacer que el workflow sea más fácil de leer y entender.
* Mejorar la "organización" del workflow.
* Incrementar la reproducibilidad del proceso.

#### Algunas definiciones

##### ``fit`` vs ``transform``

* ``fit`` encuentra los parámetros del modelo que será luego utilizado para transformar los datos. No necesariamente tiene que hacer algo más que retornar el objeto ``Transformer``, es decir, si mismo. 

* ``transform`` aplica la transformación a los datos de entrada, retornando los datos transformados.

* ``fit_transform`` aplica el ``fit`` y ``transform`` de forma consecutiva.

Cuando estamos usando un ``Pipeline``, ``fit`` y ``fit_transform`` tienen el mismo comportamiento salvo para el último elemento del pipeline. Para aquellos elementos previos al último realizan el ``fit`` de cada uno de los elementos del pipe y luego sus correspondientes ``transform``, mientras que para el último, el ``fit`` invoca al ``fit`` y el ``fit_transform`` al ``fit_transform``. 

En el caso de los estimadores, también tendremos el ``predict`` que aplica los ``transform`` sobre los datos y luego realiza el ``predict`` sobre el último elemento del pipeline.

Como ya hemos visto, puede suceder que ustedes quieran seleccionar atributos numéricos o categóricos para representar los comentarios. Para esos casos, también pueden incorporar el procesamiento dentro del pipeline.

Sklearn provee funciones para las transformaciones más comunes, las cuales se pueden encontrar en el paquete [preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing).

Vamos a ejemplificar el uso de estos transformers con un dataset sencillo que cuenta con datos numéricos y categóricos.

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv"

df = pd.read_csv(url)
df = df[:20]

print(df)

Una vez que el dataset está cargado, vamos a utilizar la función ``dtypes`` de pandas para obtener un poquito más de información acerca de las features.

In [ ]:
df.dtypes

Como se puede observar, el dataset tiene features tanto categóricos como numéricos, con lo que al menos vamos a tener que aplicar un one-hot encoding y alguna forma de escalado.

Para ello, vamos a utilizar un pipeline de los ``sklearn`` para realizar esas transformaciones y, al mismo tiempo vamos a aplicar el método ``fit`` del modelo.

Antes de construir el pipeline, vamos a dividir el conjunto de datos en la parte de training y la de test.

Recordemos que luego para hacer el ``fit`` del modelo tenemos que quitar del conjunto de training la feature objetivo (es decir, la que queremos estimar). Esto puede hacerse ahora, o puede hacerse después justo antes de entrenar el modelo.

In [ ]:
from sklearn.model_selection import train_test_split

#df_prepared = df.drop("charges", axis=1) # todas estas líneas comentadas sacan la característica a predecir
# separating the values to be predicted (labels)
#df_labels = df["charges"].copy()

#X_train, X_test, y_train, y_test = train_test_split(df_prepared, df_labels, test_size=0.2)

train_set,test_set = train_test_split(df,test_size=0.2,random_state = 34) # acá todavía dejamos la características a predecir

El primer paso para construir el pipeline es definir los tipos de los transformers. La convención que vamos a seguir es la de crear diferentes transformers para cada uno de los tipos de features.



Entonces, primero vamos a crear un transformer que aplica un escalado (```StandardScaler```) e incluye un ```SimpleImputer``` para completar los valores faltantes.

Existen diferentes opciones para completar los valores faltantes. En este caso, vamos a utilizar la media. Recordar que puede ser que la media no sea el método que nos permita obtener los mejores resultados.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

En el caso de un transformer para una feature categórica, también se puede utilizar un ```SimpleImputer```, pero en este caso con otro método que define cómo completar. Acá vamos a usar uno que simplemente nos completa cada valor faltante con la palabra "missing". Luego, vamos a aplicar un ```OneHotEncoder```para transformar los valores categóricos en enteros. Recordemos que los modelos que queremos entrenar "no se llevan bien" con atributos que no esten representados con números.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

Luego, vamos a usar el ```ColumnTransformer``` para aplicar las transformaciones definidas a las columnas correctas del dataframe. Es decir, que las transformaciones para atributos numéricos sean aplicadas solo a las columnas que son numéricas. 

Para ello, vamos a guardarnos en una lista las columnas correspondientes a cada tipo. Noten el cuidado de eliminar la columna que representa lo que vamos a estimar, dado que no tiene sentido aplicarle transformaciones.

Por otra parte, las columnas las obtenemos de la partición de training. Se podría haber utilizado el dataset completo o la partición de test. Todas tienen las mismas features.

In [ ]:
numeric_features = train_set.select_dtypes(include=['int64', 'float64']).drop(['charges'],axis=1).columns
categorical_features = train_set.select_dtypes(include=['object']).columns

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

El próximo paso es crear el pipeline que combinar el pre-proceso con el regresor. En este caso, elegimos para empezar un ```RandomForestRegressor```.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', RandomForestRegressor())])

Luego, ya podemos invocar el método ```fit``` sobre los datos crudos y eso lo que hará será: 

1. Aplicar el pre-procesamiento.
2. Entrenar el modelo.

In [ ]:
X_train = train_set.drop("charges", axis=1) # sacamos la columna
y_train = train_set["charges"].copy()
rf.fit(X_train, y_train)

In [ ]:
print(rf.predict(test_set.drop('charges',axis=1)))
print(test_set['charges'])

### Creando nuestros propios transformers

En el ejemplo anterior, usamos los transformers que ya vienen por defecto en ``sklearn``, pero también podríamos querer crear nuestro propio transformer.

Para esto, debemos crear una nueva clase que extienda los ``Transformers`` para el comportamiento que nosotros queremos. En este sentido, tenemos que implementar algunos métodos:

* ``fit``. Preparar el modelo interno de nuestro ``Transformer``. Puede que no necesiten hacer nada. 
* ``inverse_transform``. Cuál es el resultado de deshacer la transformación? No siempre se puede implementar.
* ``tranform``. El procesamiento propiamente dicho que queremos aplicar. 

La complicación que tiene esta implementación es el tipo de datos que se esperan que se retornen. En principio, la salida de esto va directo al modelo a entrenar, por lo que tenemos que retornar aquello que espera dicho ``Transformer``: una representación matricial de nuestros datos.

Para nuestro ejemplo, vamos a re-implementar una simplificación del [``MinMaxScaler``](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html?highlight=minmaxscaler) transformer.

Esta transformación está dada por:
```
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min
```

Vamos por partes.

1. **Definimos la clase y el constructor:**

La clase va a ser una extensión de ``BaseEstimator`` que es la base de los transformers en ``sklearn``. En nuestro constructor vamos a aceptar un mapa variable de parámetros, del que solo nos importa el ``feature_range`` que nos define el rango en el cual queremos hacer la normalización.


```
class CustomMinMaxScaler(BaseEstimator):
  
    def __init__(self, **params): # solo necesitamos saber a qué rango debemos hacer la normalización, por defecto a 0,1
        self.feature_range = params.get('feature_range',(0,1))
        
```

2. **Definimos el ``fit``.**

El ``fit`` nos define/calcula los "parámetros" sobre los cuales se hará luego la transformación. En este caso, hace el cálculo del rango. Si bien por parámetro puede recibir las clases (``y``), para este tipo de transformers no es necesario.

Se debe retornar a si mismo.

```
    def fit(self, X, y=None): 
        
        if self.feature_range[0] >= self.feature_range[1]: # si no hay rango positivo para la normalización no podemos trabajar
            raise ValueError("Minimum of desired feature range must be smaller"
                             " than maximum. Got %s." % str(feature_range))
        
        data_min = np.nanmin(X, axis=0)
        data_max = np.nanmax(X, axis=0)
        data_range = data_max - data_min
        
        self.scale_ = ((self.feature_range[1] - self.feature_range[0]) / data_range)
                    
        self.min_ = self.feature_range[0] - data_min * self.scale_
        self.data_min_ = data_min
        self.data_max_ = data_max
        self.data_range_ = data_range
        
        return self
```

3. **Definimos el ``transform``.**

Cómo tiene nuestro transformer que transformar nuestra entrada? Acá implementamos la función de normalización, tal como la definimos previamente. 

Notar que esta transformación solo puede ser aplicada sobre datos numéricos y que al no hacer ningún controlor sobre el tipo de datos de las columnas estamos asumiendo que ``X`` solo contendrá datos numéricos.


```
    def transform(self, X): # no restringe a qué tipo de datos acepta, siempre y cuando se pueda operar con ellos
 
        X *= self.scale_
        X += self.min_

        return X
```

4. **Definimos el ``fit_transform``.**

No tiene nada de "especial". Es solo la invocación consecutiva del ``fit`` y el ``transform``.

```
    def fit_transform(self, X, y=None):
        self.fit(X,y)
        return self.transform(X)
    
```

5. **Definimos el ``inverse_transform``.**

Cómo podemos deshacer la transformación? En este caso, aplicando las operaciones matemáticas inversas a las utilizadas en el ``transform``.

```
    def inverse_transform(self, X, y=None): # en este caso es posible hacer la reconstrucción del valor original
        
        X -= self.min_
        X /= self.scale_
                       
        return X

```

Ahora todo el código junto!

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class CustomMinMaxScaler(BaseEstimator):
  
    def __init__(self, **params): # solo necesitamos saber a qué rango debemos hacer la normalización, por defecto a 0,1
        self.feature_range = params.get('feature_range',(0,1))
        
    def fit(self, X, y=None): 
        
        if self.feature_range[0] >= self.feature_range[1]: # si no hay rango positivo para la normalización no podemos trabajar
            raise ValueError("Minimum of desired feature range must be smaller"
                             " than maximum. Got %s." % str(feature_range))
        
        data_min = np.nanmin(X, axis=0)
        data_max = np.nanmax(X, axis=0)
        data_range = data_max - data_min
        
        self.scale_ = ((self.feature_range[1] - self.feature_range[0]) / data_range)
                    
        self.min_ = self.feature_range[0] - data_min * self.scale_
        self.data_min_ = data_min
        self.data_max_ = data_max
        self.data_range_ = data_range
        
        return self

    def transform(self, X): # no restringe a qué tipo de datos acepta, siempre y cuando se pueda operar con ellos
 
        X *= self.scale_
        X += self.min_

        return X

    def fit_transform(self, X, y=None):
        self.fit(X,y)
        return self.transform(X)
    
    def inverse_transform(self, X, y=None): # en este caso es posible hacer la reconstrucción del valor original
        
        X -= self.min_
        X /= self.scale_
                       
        return X

Ahora que ya lo tenemos implementado, lo podemos usar. Notar que seleccionamos las columnas numéricas al pasar nuestro ``df`` para evitar errores.

In [ ]:
CustomMinMaxScaler().fit_transform(train_set[numeric_features])

Para comprobar que los resultados que obtuvimos son los correctos, podemos utilizar el transformer ``MinMaxScaler`` de sklearn y controlar que los resultados sean los mismos.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

MinMaxScaler().fit_transform(train_set[numeric_features])

Listo. Ahora este transformer lo podemos utilizar en los mismos lugares que utilizamos los transformers pre-definidos en ``sklearn``.

#### Ejercicios

1. Reimplementar el [``SimpleImputer``](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html).

In [ ]:
# TODO

2. El ``MinMaxScaler`` solo puede ser aplicado sobre columnas numéricas. En nuestra implementación de ejemplo no hicimos ningún control acerca del tipo de las columnas en el ``X`` que recibimos, asumiendo que solo se va a invocar el transformer sobre columnas numéricas. Agregar el código necesario para controlar que el transformer solo se aplique a columnas numéricas de la entrada e ignore el resto.

In [ ]:
# TODO